# 🔥 프로젝트: 멋진 작사가 만들기


## Step 1. 데이터 다운로드

> https://aiffelstaticprd.blob.core.windows.net/media/documents/song_lyrics.zip  <br/>
  song_lyrics 데이터 받고 압축풀기

---

## Step 2. 데이터 읽어오기

- **`glob`** 모듈: 파일을 읽어오는 작업을 하기가 아주 용이 
- 모든 txt 파일을 읽어온 후, raw_corpus 리스트에 문장 단위로 저장

In [1]:
import glob
import os

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'
txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 모든 txt 파일을 모두 읽고 raw_corpus에 문장단위로 저장.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:9])
print(len(raw_corpus[3]))

데이터 크기: 187088
Examples:
 ['THE QUEEN _of_ HEARTS', '', '', '    The Queen of Hearts she made some tarts,', "      All on a summer's day;", '', '    The Knave of Hearts he stole those tarts,', '      And took them clean away.', '']
44


## Step 3. 데이터 정제

- `preprocess_sentence()` 함수를 만들어 데이터를 정제


- 지나치게 긴 문장은 다른 데이터들이 과도한 Padding을 갖게 하므로 제거 
- 너무 긴 문장은 노래가사 작사하기에 어울리지 않을수도 있으므로(문장을 토큰화 했을 때 토큰의 개수가 15개를 넘어가는 문장을 학습데이터에서 제외)



* re 참고자료: https://greeksharifa.github.io/%EC%A0%95%EA%B7%9C%ED%91%9C%ED%98%84%EC%8B%9D(re)/2018/07/20/regex-usage-01-basic/

In [2]:
import re                  # 정규표현식(문장 데이터를 정돈하기 위해) 
import numpy as np         # 변환된 문장 데이터(행렬)을 편하게 처리하기 위해
import tensorflow as tf  

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제

    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 변환
    # 1. 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)
    # 2. 공백 패턴 -> '스페이스 1개로 치환'(지우겠다)
    sentence = re.sub(r'[" "]+', " ", sentence)  
    # 3. a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도) -> '스페이스 1개로 치환'        
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)
    
    sentence = sentence.strip() # 공백으로 바뀐부분 다 지우기

    sentence = '<start> ' + sentence + ' <end>'      

    return sentence

In [3]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue


    corpus.append(preprocess_sentence(sentence))

print("데이터 크기:", len(corpus))
print("Examples:\n", corpus[:3])

데이터 크기: 175749
Examples:
 ['<start> the queen of hearts <end>', '<start> the queen of hearts she made some tarts , <end>', '<start> all on a summer s day <end>']


## Step 4. 평가 데이터셋 분리

- `tokenize()` 함수로 데이터를 Tensor로 변환한 후, 


- `sklearn` 모듈의 `train_test_split()` 함수를 사용해 훈련 데이터와 평가 데이터를 분리


- **단어장의 크기는 12,000 이상**으로 설정하세요! 


- **총 데이터의 20%를 평가 데이터셋으로 사용**해 주세요!

> enc_train, enc_val, dec_train, dec_val = <코드 작성>

여기까지 올바르게 진행했을 경우, 아래 실행 결과를 확인할 수 있습니다.

---

>print("Source Train:", enc_train.shape) <br/>
print("Target Train:", dec_train.shape)

> out:
>> Source Train: (124960, 14)
>>Target Train: (124960, 14)

---

- 만약 결과가 다르다면 천천히 과정을 다시 살펴 동일한 결과를 얻도록 하세요! 

- 만약 학습데이터 갯수가 124960보다 크다면 위 Step 3.의 데이터 정제 과정을 다시한번 검토

In [4]:
def tokenize(corpus):

    # <Tokenizer 패키지 생성 by tensorflow>
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=15000,  # 전체 단어의 개수 
        filters=' ',      # 별도로 전처리 로직을 추가할 수 있지만 지금은 X
        oov_token="<unk>"  # 사전에 없었던 단어 -> 어떤 토큰으로 대체할지 설정
    )
    
    # 만들어 놓은 corpus로부터 Tokenizer가 <사전>을 자동생성
    tokenizer.fit_on_texts(corpus)   

    # 이후 tokenizer를 활용하여 모델에 입력할 <데이터셋을 구축>
    # tokenizer는 구축한 사전으로부터 / corpus를 해석해 / Tensor로 변환
    tensor = tokenizer.texts_to_sequences(corpus)   
    
    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 <padding 메소드>를 제공
    # maxlen의 디폴트값= None (corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰짐)
    # 즉, post: 뒷쪽의 남은 공간에 0이 추가되어 길이가 맞춰짐.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  

    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

tensor.shape

[[  2   6 818 ...   0   0   0]
 [  2   6 818 ...   0   0   0]
 [  2  24  18 ...   0   0   0]
 ...
 [  2   5  90 ...   0   0   0]
 [  2   9 157 ...   0   0   0]
 [  2 160  15 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7fd2c3910ac8>


(175749, 347)

In [5]:
# <생성된 단어 사전 확인>
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])
    
    if idx >= 10: break
        
print(len(tokenizer.index_word))

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to
27592


In [13]:
# tensor에서 마지막 토큰을 잘라내서 [소스 문장 = data]을 생성합니다. 
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  

# tensor에서 <start>를 잘라내서 [타겟 문장 = label]을 생성합니다.
tgt_input = tensor[:, 1:]    

print('Source Sentence: ',src_input[0].shape) # data
print('Target Sentence: ',tgt_input[0].shape) # label

Source Sentence:  (346,)
Target Sentence:  (346,)


In [7]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val  = train_test_split(src_input, 
                                                            tgt_input, 
                                                            test_size=0.2,
                                                            shuffle=True)
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (140599, 346)
Target Train: (140599, 346)


In [8]:
# source / target

# tensor에서 마지막 토큰 자름 -> [소스 문장]을 생성
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높음.
enc_train = tensor[:, :-1]  

# tensor에서 <start>를 자름 -> [타겟 문장]을 생성
dec_train = tensor[:, 1:]    

print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (175749, 346)
Target Train: (175749, 346)


In [9]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

# tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
VOCAB_SIZE = tokenizer.num_words + 1    

# 🌟tf.data.Dataset객체 생성
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 346), (256, 346)), types: (tf.int32, tf.int32)>

## Step 5. 인공지능 만들기

모델의 Embedding Size와 Hidden Size를 조절하며 

- 10 Epoch 안에 `val_loss` 값을 2.2 수준으로 줄일 수 있는 모델을 설계하세요! (Loss는 아래 제시된 Loss 함수를 그대로 사용!)

> #Loss  <br/>
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

>generate_text(lyricist, tokenizer, init_sentence = " `<start>` i love ", max_len=20)

그리고 멋진 모델이 생성한 가사 한 줄을 제출하시길 바랍니다!


In [10]:
class TextGenerator(tf.keras.Model):

    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()

        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)

    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)

        return out

embedding_size = 256
hidden_size = 1024

model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [11]:
model.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or calling `fit()` with some data, or specify an `input_shape` argument in the first layer(s) for automatic build.

In [ ]:
optimizer = tf.keras.optimizers.Adam()

loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)

model.fit(dataset, epochs=5)

In [ ]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):

    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]  

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, 
                                 tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  
        # while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""

    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 
    # 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

In [ ]:
generate_text(lyricist, tokenizer, init_sentence = " <start> i love ", max_len=20)

# 😭 이번주에 이사 준비로 너무 바빴습니다 ㅠㅠ 
# 😭😭주말까지 해서 다시 업로드시키겠습니다 ㅠㅠ

## [ 루브릭 ]

1. 가사 텍스트 생성 모델이 정상적으로 동작하는가?  <br/>
	텍스트 제너레이션 결과가 그럴듯한 문장으로 생성되는가?
    

2. 데이터의 전처리와 데이터셋 구성 과정이 체계적으로 진행되었는가?  <br/>
	특수문자 제거, 토크나이저 생성, 패딩처리 등의 과정이 빠짐없이 진행되었는가?
    

3. 텍스트 생성모델이 안정적으로 학습되었는가?  <br/>
	텍스트 생성모델의 validation loss가 2.2 이하로 낮아졌는가?

## [ 회 고 ]

1. 이번 프로젝트에서 어려웠던 점: 


2. 프로젝트를 진행하면서 알아낸 점 혹은 아직 모호한 점.


- 알아낸 점: 


- 모호한 점: 


3. 루브릭 평가 지표를 맞추기 위해 시도한 것들.
  - 